In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from datetime import date

In [5]:
today = date.today()
print(today, type(today))
today = str(today)
# print(today, type(today))

2024-09-10 <class 'datetime.date'>


In [6]:
page = 1
page_size = 100
final_result_df = pd.DataFrame()
while True:
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method='searchCorpList',pageIndex=page, currentPageSize=page_size,orderMode=3,orderStat='D',searchType=13, fiscalYearEnd='all', location='all')
    r = requests.get(url, params=payload)
    print(r.status_code, end="\r")
    soup = bs(r.text, 'lxml')
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",", ""))
    total_pages = total_items // page_size + 1
    print(f"{page}/{total_pages} 수집중", end="\r")
    keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")  
    result = {}
    for tr in soup.select('tr'):
        for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
            if idx == 0:
                kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
                kind = ", ".join(kinds)
                code = td.select_one('a')['onclick'].split("'")[1]+"0" # 종목코드 추출
                result.setdefault('증권종류', []).append(kind) # 증권종류 저장
                result.setdefault(key, []).append(td.text)   # 회사이름 저장
                result.setdefault('종목코드', []).append(code)
            elif idx == 6:
                home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
                result.setdefault(key, []).append(home_link)
            else:
                result.setdefault(key, []).append(td.text)
    result_df = pd.DataFrame(result)
    final_result_df = pd.concat([final_result_df, result_df])
        
    if page < total_pages:
        page += 1
        time.sleep(5)
    else:
        break

display(final_result_df)
final_result_df.to_csv(f"상장기업정보{today}_기준.csv", encoding='utf-8', index=False)  # 수집일 기준으로 csv 파일로 저장



,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,아이비젼웍스,469750,특수 목적용 기계 제조업,이차전지 검사시스템 및 검사시스템 설치셋업용역,2024-09-03,12월,길기재,http://www.ivw.co.kr/kor/main/,대전광역시
1,코스닥,아이스크림미디어,461300,소프트웨어 개발 및 공급업,"교육출판, 커머스, 연수",2024-08-30,12월,허주환,http://www.i-screammedia.com,경기도
2,"코스닥, 투자경고종목",이엔셀,456070,기초 의약물질 제조업,"첨단바이오의약품 위탁개발생산(CDMO) 서비스 / 차세대 세포,유전자 치료제",2024-08-23,12월,장종욱,http://www.encellinc.com/,서울특별시
3,코스닥,M83,476080,"영화, 비디오물, 방송프로그램 제작 및 배급업",VFX,2024-08-22,12월,정성진..,http://m83.co.kr,서울특별시
4,코스닥,대신밸런스제18호스팩,478780,금융 지원 서비스업,금융 지원 서비스업,2024-08-22,12월,문병식,,서울특별시
...,...,...,...,...,...,...,...,...,...,...
10,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
11,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
12,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
13,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [8]:
final_result_df = pd.read_csv("./상장기업정보2024-09-10_기준.csv")
final_result_df

,증권종류,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,코스닥,아이비젼웍스,469750,특수 목적용 기계 제조업,이차전지 검사시스템 및 검사시스템 설치셋업용역,2024-09-03,12월,길기재,http://www.ivw.co.kr/kor/main/,대전광역시
1,코스닥,아이스크림미디어,461300,소프트웨어 개발 및 공급업,"교육출판, 커머스, 연수",2024-08-30,12월,허주환,http://www.i-screammedia.com,경기도
2,"코스닥, 투자경고종목",이엔셀,456070,기초 의약물질 제조업,"첨단바이오의약품 위탁개발생산(CDMO) 서비스 / 차세대 세포,유전자 치료제",2024-08-23,12월,장종욱,http://www.encellinc.com/,서울특별시
3,코스닥,M83,476080,"영화, 비디오물, 방송프로그램 제작 및 배급업",VFX,2024-08-22,12월,정성진..,http://m83.co.kr,서울특별시
4,코스닥,대신밸런스제18호스팩,478780,금융 지원 서비스업,금융 지원 서비스업,2024-08-22,12월,문병식,NaN,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2710,"유가증권, KTOP30, KOSPI200, KRX300",유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2711,"유가증권, KOSPI200, KRX300",CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2712,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2713,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


In [11]:
!conda install PyMySQL       # mysql에 접속하게 해주는 라이브러리
!conda install sqlalchemy    # sql을 쉽게 사용하게 해주는 라이브러리

Retrieving notices: ...working... done



CondaValueError: invalid package specification: #


CondaValueError: invalid package specification: #



In [28]:
import os
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()

In [ ]:
def db_connect():
    engine = create_engine("%s+%s://%s:%s@%s/%s" % (db, dbtype, id, pw, host, database))
    conn = engine.connect()
    return conn

In [ ]:
# DB에 접속해서 저장하는 함수
def exi_to_db(table_name, date, df):
    conn = db_connect()    
    df.to_sql(table_name , con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{table_name}_{date}, {'저장완료':<30s}", end="\r")

# 네이버 뉴스 DB저장
def news_to_db(table_name, date, df):
    conn = news_db_connet()    
    df.to_sql(table_name , con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{table_name}_{date}, {'저장완료':<30s}", end="\r")

In [23]:
import os
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()

In [25]:
# MySQL 접속 정보 설정
dbtype = "pymysql"
user = "root"
password = "1234"
host = "127.0.0.1:3306"
database = "korea_stock"  # 사용할 데이터베이스 이름

In [26]:
# # MySQL 접속 정보 설정
# dbtype = "pymysql"
# user = "root"
# password = "1234"
# host = "127.0.0.1:3306"
# database = "korea_stock"  # 사용할 데이터베이스 이름

# MySQL 서버에 연결 (데이터베이스를 지정하지 않고 접속)
engine = create_engine(f"mysql+{dbtype}://{user}:{password}@{host}")

# MySQL 연결 열기 및 데이터베이스 생성
with engine.connect() as conn:
    # 데이터베이스가 존재하지 않으면 생성
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database};"))
    print(f"'{database}' 데이터베이스가 성공적으로 생성되었거나 이미 존재합니다.")

# 데이터베이스와 연결을 다시 시도
engine = create_engine(f"mysql+{dbtype}://{user}:{password}@{host}/{database}")
conn = engine.connect()

# 연결 성공 여부 확인
print("Connected to the database.")

'korea_stock' 데이터베이스가 성공적으로 생성되었거나 이미 존재합니다.
Connected to the database.


In [27]:
#DataFrame을 MySQL 테이블로 저장
engine = create_engine(f"mysql+{dbtype}://{user}:{password}@{host}/{database}")
conn = engine.connect()
final_result_df.to_sql(f'stock_company_info_<서민서>', con=conn, if_exists='replace', index=False)
conn.close()

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from datetime import date
import os
from sqlalchemy import create_engine, text
import pymysql

# 오늘 날짜 설정
today = date.today()
today = str(today)

# MySQL 접속 정보 설정
dbtype = "pymysql"
user = "root"
password = "1234"
host = "127.0.0.1:3306"
database = "korea_stock"  # 사용할 데이터베이스 이름


# 데이터 수집 부분 (기존 코드 사용)
page = 1
page_size = 100
final_result_df = pd.DataFrame()
while True:
    url = "https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method='searchCorpList', pageIndex=page, currentPageSize=page_size, orderMode=3, orderStat='D', searchType=13, fiscalYearEnd='all', location='all')
    r = requests.get(url, params=payload)
    print(r.status_code, end="\r")
    soup = bs(r.text, 'lxml')
    total_items = int(soup.select_one(".info.type-00 > em").text.replace(",", ""))
    total_pages = total_items // page_size + 1
    print(f"{page}/{total_pages} 수집중", end="\r")
    keys = soup.select_one("table.list.type-00.tmt30")['summary'].split(", ")  
    result = {}
    for tr in soup.select('tr'):
        for idx, (key, td) in enumerate(zip(keys, tr.select('td'))):
            if idx == 0:
                kinds = [img['alt'].strip() for img in td.select('img')]   # 1번째 증권 종류, 회사이름
                kind = ", ".join(kinds)
                code = td.select_one('a')['onclick'].split("'")[1]+"0" # 종목코드 추출
                result.setdefault('증권종류', []).append(kind) # 증권종류 저장
                result.setdefault(key, []).append(td.text)   # 회사이름 저장
                result.setdefault('종목코드', []).append(code)
            elif idx == 6:
                home_link = td.select_one('a')['href'] if td.string == None else ""  # 6번째 링크 찾기
                result.setdefault(key, []).append(home_link)
            else:
                result.setdefault(key, []).append(td.text)
    result_df = pd.DataFrame(result)
    final_result_df = pd.concat([final_result_df, result_df])
        
    if page < total_pages:
        page += 1
        time.sleep(5)
    else:
        break

# 데이터 확인
print(final_result_df)

# CSV로 저장
final_result_df.to_csv(f"상장기업정보_{today}기준.csv", encoding='utf-8', index=False)

# 이미 저장된 CSV 파일 불러오기
final_result_df = pd.read_csv(f"./상장기업정보_{today}기준.csv")

# MySQL 연결 설정
pymysql.install_as_MySQLdb()

# MySQL 서버에 연결 (데이터베이스를 지정하지 않고 접속)
engine = create_engine(f"mysql+{dbtype}://{user}:{password}@{host}")

# MySQL 연결 열기 및 데이터베이스 생성
with engine.connect() as conn:
    # 데이터베이스가 존재하지 않으면 생성
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database};"))
    print(f"'{database}' 데이터베이스가 성공적으로 생성되었거나 이미 존재합니다.")

# 데이터베이스와 연결을 다시 시도
engine = create_engine(f"mysql+{dbtype}://{user}:{password}@{host}/{database}")
conn = engine.connect()

# 연결 성공 여부 확인
print("Connected to the database.")

# DataFrame을 MySQL 테이블로 저장
table_name = f"company_info"

# MySQL로 데이터 저장
final_result_df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

# 연결 닫기
conn.close()

print(f"데이터가 MySQL의 '{table_name}' 테이블에 성공적으로 저장되었습니다.")
